In [1]:
#!pip install pyyaml==5.1
#!pip install labelme
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

torch:  1.8 ; cuda:  cu111


In [2]:
# import some common libraries
import random
import cv2
import json
import os
import numpy as np
import argparse
import torch
import detectron2
import glob
# import some common detectron2 utilities
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import pascal_voc, register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, PascalVOCDetectionEvaluator
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger


In [3]:
# 准备数据集合训练代码
#!rm -r detectron2electric/
#!git clone https://github.com/DucLune/detectron2electric.git
#!cp /content/detectron2electric/traincode/* ./ -rf

In [4]:
# Some basic setup:
# Setup detectron2 logger
setup_logger()

# 创建解析
parser = argparse.ArgumentParser(
    description="detectron2 demo", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# 添加参数
# 使用Cloab等平台时请注意设置正确的root_dir为detectron2electric的路径
parser.add_argument('--root_dir', type=str,
                    default="/root/public_data/ElectricalComponent-MaskRCNN/detectron2electric")
parser.add_argument('--train_url', type=str,
                    default="model", help='the path model saved')
parser.add_argument('--dataset', type=str, default='coco',
                    help='the dataset dirname')
parser.add_argument('--device', type=str, default='cuda',
                    help='the training device')
parser.add_argument('--num_classes', type=int, default=3,
                    help='cfg.MODEL.ROI_HEADS.NUM_CLASSES ')

# 解析参数
args, unkown = parser.parse_known_args()
os.chdir(args.root_dir)
print("setting woring path to :"+os.getcwd())
from util import pictureUtils
pictureUtils.buildCocoDataset("Annotations",os.path.join(args.dataset),["glue","injection_hole","pin_glue"],0.7,0.3,0)
register_coco_instances("mydataset_train", {}, os.path.join(args.dataset,"annotations","train.json"),"SmallJPGImages")
register_coco_instances("mydataset_val", {}, os.path.join(args.dataset,"annotations","val.json"),"SmallJPGImages" )

setting woring path to :/root/public_data/ElectricalComponent-MaskRCNN/detectron2electric


In [5]:
# %%
# 定义模型并训练
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = os.path.join("../weights/model_final_a3ec72.pkl")
# cfg.MODEL.WEIGHTS = os.path.join(args.train_url, "model_final.pth")
cfg.MODEL.DEVICE = args.device
cfg.MODEL.RPN.NMS_THRESH = 0.7
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
cfg.DATASETS.TRAIN = ("mydataset_train",)
cfg.DATASETS.TEST = ("mydataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.MAX_ITER = 600
cfg.SOLVER.STEPS = []        # do not decay learning rate
# faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = args.num_classes
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = args.train_url
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
if args.device != 'cpu':
    trainer.train()

[INFO   ] defaults:build_model:507 - Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[12/17 12:55:17 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[INFO   ] build:filter_images_with_only_crowd_annotations:67 - Removed 3 images with no usable annotations. 61 images left.


[12/17 12:55:17 d2.data.build]: Removed 3 images with no usable annotations. 61 images left.


[INFO   ] logger:log_first_n:172 - Distribution of instances among all 3 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
|    glue    | 194          | injection_h.. | 90           |  pin_glue  | 18           |
|            |              |               |              |            |              |
|   total    | 302          |               |              |            |              |


...

[12/17 13:00:10 d2.data.common]: Serializing 28 elements to byte tensors and concatenating them all ...


[INFO   ] common:__init__:144 - Serialized dataset takes 0.04 MiB


[12/17 13:00:10 d2.data.common]: Serialized dataset takes 0.04 MiB


[WARNING] defaults:test:602 - No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


WARNING [12/17 13:00:10 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


In [8]:
# %%
# 验证集验证

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
# path to the model we just trained
# 加载训练出来的权重
cfg.MODEL.WEIGHTS = os.path.join(args.train_url, "model_final.pth")
# 构建评估器
evaluator = COCOEvaluator("mydataset_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "mydataset_val")
predictor = DefaultPredictor(cfg)
# 输出模型在验证集上的性能指标
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`
#model = trainer.build_model(cfg)
#metrics = trainer.test(cfg, model, evaluator)

[12/17 13:03:22 d2.data.datasets.coco]: Loaded 28 images in COCO format from coco/annotations/val.json


[INFO   ] dataset_mapper:__init__:83 - [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


[12/17 13:03:22 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


[INFO   ] common:__init__:135 - Serializing 28 elements to byte tensors and concatenating them all ...


[12/17 13:03:22 d2.data.common]: Serializing 28 elements to byte tensors and concatenating them all ...


...

[12/17 13:03:27 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category   | AP    | category       | AP     | category   | AP     |
|:-----------|:------|:---------------|:-------|:-----------|:-------|
| glue       | 4.149 | injection_hole | 72.797 | pin_glue   | 31.065 |
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[12/17 13:03:27 d2.evaluation.fast_eval_api]: Evaluate annotation type *segm*
[12/17 13:03:27 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[12/17 13:03:27 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[12/17 13:03:27 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.00 seconds.


[INFO   ] coco_evaluation:_derive_coco_results:340 - Evaluation results for segm: 
|   AP   |  AP50  |  AP75  |  APs   |  APm   |  APl   |
|:------:|:------:|:------:|:------:|:------:|:------:|
| 36.402 | 59.185 | 40.274 | 18.832 | 37.549 | 19.307 |


 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.364
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.193
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.231
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.436
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.439
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.447
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.200
[12/17 13:03:27 d2.evaluation.coco_evalu

[INFO   ] coco_evaluation:_derive_coco_results:374 - Per-category segm AP: 
| category   | AP    | category       | AP     | category   | AP     |
|:-----------|:------|:---------------|:-------|:-----------|:-------|
| glue       | 3.571 | injection_hole | 68.326 | pin_glue   | 37.308 |


[12/17 13:03:27 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category       | AP     | category   | AP     |
|:-----------|:------|:---------------|:-------|:-----------|:-------|
| glue       | 3.571 | injection_hole | 68.326 | pin_glue   | 37.308 |
OrderedDict([('bbox', {'AP': 36.003635137949885, 'AP50': 59.381646497983134, 'AP75': 35.14386557703389, 'APs': 23.877427350952964, 'APm': 36.905761766750004, 'APl': 21.287128712871286, 'AP-glue': 4.149182775420399, 'AP-injection_hole': 72.7967661427797, 'AP-pin_glue': 31.064956495649565}), ('segm', {'AP': 36.4017534689977, 'AP50': 59.18454345434543, 'AP75': 40.27424170988528, 'APs': 18.832006552303582, 'APm': 37.549399304867876, 'APl': 19.306930693069308, 'AP-glue': 3.570623133741946, 'AP-injection_hole': 68.32620643016683, 'AP-pin_glue': 37.30843084308431})])


In [7]:
#!rm model/*.jpg
#!mkdir model

# 在测试集上测试，并保存图片
import os
demo = VisualizationDemo(cfg)
#!rm model/*.jpg
#!mkdir model
filePath = os.path.join(args.data_url, args.dataset, "test")
#list_data = os.listdir(filePath)
list_data = [] 
for filename in list_data:
    if filename.split('.')[1] == 'json':
        continue
    im = cv2.imread(os.path.join(filePath, filename))
    predictions, visualized_output = demo.run_on_image(im)
    visualized_output.save(os.path.join(args.train_url, filename))


NameError: name 'VisualizationDemo' is not defined